In [5]:
import pandas as pd

## Data: How is the .csv organized?

For each .csv file the following apply:
* each row represents the data of a single vehicle
* the first 10 columns in the 1st row include the columns’ names
* the first 4 columns include information about the trajectory like the unique trackID, the type of vehicle, the distance traveled in meters and the average speed of the vehicle in km/h
* the last 6 columns are then repeated every 6 columns based on the time frequency. For example, column_5 contains the latitude of the vehicle at time column_10, and column­­­_11 contains the latitude of the vehicle at time column_16.
* Speed is in km/h, Longitudinal and Lateral Acceleration in m/sec2 and time in seconds.

In [8]:
with open("../Week_2/20181101_d1_0800_0830.csv", 'r') as file:
    lines = file.readlines()

In [9]:
print(f"The number of rows/lines is {len(lines)}")

The number of rows/lines is 786


In [10]:
print(lines[0]) # column names
print(lines[0].strip('\n').strip().strip(';').split(';')) # columns names as a list

track_id; type; traveled_d; avg_speed; lat; lon; speed; lon_acc; lat_acc; time

['track_id', ' type', ' traveled_d', ' avg_speed', ' lat', ' lon', ' speed', ' lon_acc', ' lat_acc', ' time']


In [11]:
lines_as_lists = [line.strip('\n').strip().strip(';').split(';') for line in lines]
len(lines_as_lists)

786

In [12]:
print(f"the number of fields in row 1 is {len(lines_as_lists[1])}, row 2 is {len(lines_as_lists[2])}")

the number of fields in row 1 is 3310, row 2 is 550


The number of fields in each row is = 4 + 6 * n

* The first 4 unique values are: track_id; type; traveled_d; avg_speed
* The remaining fields are repeated sequences of: 
  * lat_1; lon_1; speed_1; lon_acc_1; lat_acc_1; time_1
  * lat_2; lon_2; speed_2; lon_acc_2; lat_acc_2; time_2
  * ...
  * lat_n; lon_n; speed_n; lon_acc_n; lat_acc_n; time_n

* n depends on the time frequency. It is different for each row.


In [13]:
no_field_max = 0

for row in lines_as_lists:
    if len(row) > no_field_max:
        no_field_max = len(row)

print(f"the maximum number of fields is {no_field_max}")
largest_n = int((no_field_max-4)/6)
print(f"the largest n = {largest_n}")

the maximum number of fields is 121810
the largest n = 20301


We can divide each row into 2 parts:
1.  A list of the first 4 values. 
    - These will correspond to ['track_id', ' type', ' traveled_d', ' avg_speed']
2.  A matrix of size 6*n of the remaining values. 
    - These will correspond to a table with columns [' lat', ' lon', ' speed', ' lon_acc', ' lat_acc', ' time'] and n rows.
    - We will append the 'track_id' value to each row to keep track of the vehicle identity.

In [14]:
cols = lines_as_lists.pop(0)

In [15]:
track_cols = cols[:4]
trajectory_cols = ['track_id'] + cols[4:]

print(track_cols)
print(trajectory_cols)


['track_id', ' type', ' traveled_d', ' avg_speed']
['track_id', ' lat', ' lon', ' speed', ' lon_acc', ' lat_acc', ' time']


In [16]:
track_info = []
trajectory_info = []

for row in lines_as_lists:
    track_id = row[0]

    # add the first 4 values to track_info
    track_info.append(row[:4]) 

    remaining_values = row[4:]
    # reshape the list into a matrix and add track_id
    trajectory_matrix = [ [track_id] + remaining_values[i:i+6] for i in range(0,len(remaining_values),6)]
    # add the matrix rows to trajectory_info
    trajectory_info = trajectory_info + trajectory_matrix


In [17]:
df_vehicle = pd.DataFrame(data= track_info,columns=track_cols)

df_vehicle.head()

,track_id,type,traveled_d,avg_speed
0,1,Motorcycle,107.61,17.608581
1,2,Car,20.82,20.820484
2,3,Car,66.16,13.380815
3,4,Motorcycle,9.36,33.685504
4,5,Motorcycle,38.18,31.240518


In [18]:
#Finding the % of missing values in each column
# Step 1: Create a boolean mask for missing values
missing_values_mask = df_vehicle.isna()

# Step 2: Count the number of missing values in each column
missing_values_count = missing_values_mask.sum()

# Step 3: Calculate the percentage of missing values
total_rows = len(df_vehicle)
missing_values_percentage = (missing_values_count / total_rows) * 100

# Step 4: Print or display the results
print("Percentage of missing values in each column:")
print (missing_values_percentage)

Percentage of missing values in each column:
track_id       0.0
 type          0.0
 traveled_d    0.0
 avg_speed     0.0
dtype: float64


In [19]:
df_trajectory = pd.DataFrame(data= trajectory_info,columns=trajectory_cols)

df_trajectory.head()

,track_id,lat,lon,speed,lon_acc,lat_acc,time
0,1,37.977484,23.737106,13.4787,0.0466,-0.0660,38.400000
1,1,37.977484,23.737107,13.4811,0.0383,-0.0711,38.440000
2,1,37.977485,23.737109,13.4826,0.0329,-0.0761,38.480000
3,1,37.977485,23.737110,13.4836,0.0298,-0.0810,38.520000
4,1,37.977486,23.737112,13.4843,0.0247,-0.0860,38.560000


In [20]:
#Finding the % of missing values in each column
# Step 1: Create a boolean mask for missing values
missing_values_mask = df_trajectory.isna()

# Step 2: Count the number of missing values in each column
missing_values_count = missing_values_mask.sum()

# Step 3: Calculate the percentage of missing values
total_rows = len(df_trajectory)
missing_values_percentage = (missing_values_count / total_rows) * 100

# Step 4: Print or display the results
print("Percentage of missing values in each column:")
print (missing_values_percentage)

Percentage of missing values in each column:
track_id    0.0
 lat        0.0
 lon        0.0
 speed      0.0
 lon_acc    0.0
 lat_acc    0.0
 time       0.0
dtype: float64


In [21]:
from sqlalchemy import create_engine
import urllib.parse
encoded_password = urllib.parse.quote("Sheila@49")
connection_params = {
    "host": "localhost",
    "user": "postgres",
    "password": encoded_password,
    "port": "5432",
    "database": "TrafficDWH"
}
# Establish connection to the PostgreSQL database
engine = create_engine(
    f"postgresql+psycopg2://{connection_params['user']}:{connection_params['password']}"
    f"@{connection_params['host']}:{connection_params['port']}/{connection_params['database']}"
)
# Load df1 into the PostgreSQL database
df_vehicle.to_sql('Vehicle_Information', engine, if_exists='replace', index=False)

# Load df2 into the PostgreSQL database
df_trajectory.to_sql('Trajectory_Data', engine, if_exists='replace', index=False)





475

In [22]:
# check if every id (evry timed data) is included in the former dataframe

df_vehicle.shape[0]==len(df_trajectory.track_id.unique())

True